In [240]:
import datascience
import pandas as pd
import geoplot
import geopandas
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import numpy as np
data = pd.read_csv("indicators.csv")
print("Read Successful")


Read Successful


In [278]:
questionsDF = pd.read_csv("questions2.csv")
questionsDF.observation_units.unique()

array(['%'], dtype=object)

In [241]:
indicators = Table().read_table("indicators.csv")

In [271]:
ret = Table().with_columns("year", 0, "question", 0, "mean_val", 0).where("year", -11)
ret

year,question,mean_val


In [273]:

for question in questionsDF.predictor_question.unique().tolist():
    for year in questionsDF.year.unique().tolist():
        tempDF = questionsDF.loc[(questionsDF.predictor_question == question) & (questionsDF.year == year)].predictor_value.tolist()
        mean = 0
        for i in tempDF:
            mean += i
        mean /= len(tempDF)
        temp = Table().with_columns("year", year, "question", question, "mean_val", mean)
        ret = ret.append(temp)
        

ret.show()
ret.to_csv("arr2.csv")
        
        
        
        

year,question,mean_val
2012,Healthy weight among adults aged >= 18 years,34.5902
2011,Healthy weight among adults aged >= 18 years,35.0892
2015,Healthy weight among adults aged >= 18 years,33.5088
2013,Healthy weight among adults aged >= 18 years,34.101
2014,Healthy weight among adults aged >= 18 years,33.7304
2016,Healthy weight among adults aged >= 18 years,33.1029
2012,Overweight or obesity among women aged 18-44 years,50.9647
2011,Overweight or obesity among women aged 18-44 years,50.4039
2015,Overweight or obesity among women aged 18-44 years,52.6902
2013,Overweight or obesity among women aged 18-44 years,51.6157


In [279]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
for sentence in data.loc[data.Topic == "Diabetes"].Question.unique().tolist():
    ss = sid.polarity_scores(sentence)
    print(sentence)
    print(ss)
    print()

datas = data.loc[(data.Topic == "Diabetes") & (data.StratificationCategory1 == "Overall")].Question.unique().tolist()


sentiment_data = {}
for sentence in data.Question.unique().tolist():
    ss = sid.polarity_scores(sentence)
    tmp = data.loc[data.Question == sentence].Topic.unique().tolist()
    for topic in tmp:
        if not state in sentiment_data:
            sentiment_data[topic] = 0
        sentiment_data[topic] += ss['compound']
        
sentiment_data
    
    

Foot examination among adults aged >= 18 years with diagnosed diabetes
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Mortality due to diabetes reported as any listed cause of death
{'neg': 0.281, 'neu': 0.719, 'pos': 0.0, 'compound': -0.5994}

Mortality with diabetic ketoacidosis reported as any listed cause of death
{'neg': 0.281, 'neu': 0.719, 'pos': 0.0, 'compound': -0.5994}

Influenza vaccination among noninstitutionalized adults aged 18-64 years with diagnosed diabetes
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Diabetes prevalence among women aged 18-44 years
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Adults with diagnosed diabetes aged >= 18 years who have taken a diabetes self-management course
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Prevalence of diagnosed diabetes among adults aged >= 18 years
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

Dilated eye examination among adults aged >= 18 years with diagnosed diabetes
{'neg

{'Alcohol': -0.2023,
 'Nutrition, Physical Activity, and Weight Status': 0.4939,
 'Cardiovascular Disease': 0.0,
 'Arthritis': -0.296,
 'Diabetes': 0.0,
 'Tobacco': 0.25,
 'Overarching Conditions': -0.5106,
 'Chronic Obstructive Pulmonary Disease': -0.3818,
 'Cancer': -0.6597,
 'Chronic Kidney Disease': 0.0,
 'Asthma': 0.0,
 'Disability': 0.0,
 'Oral Health': 0.0,
 'Older Adults': 0.0,
 'Mental Health': -0.3818,
 'Immunization': 0.0,
 'Reproductive Health': 0.0}

In [229]:
set_of_states = data.LocationAbbr.unique().tolist()
set_of_states

set_of_questions = data.Question.unique().tolist()

res = []
for question in set_of_questions:
    boolean = True
    for state in set_of_states:
        tmp = data.loc[(data.Question == question) & (data.LocationAbbr == state)].Question.tolist()
        if len(tmp) == 0:
            boolean = False
            break
    if boolean:
        res.append(question)
res
            

TypeError: descriptor 'remove' requires a 'set' object but received a 'str'

In [228]:
len(res)

len(set_of_questions)

for state in set_of_states:
    print(len(data.loc[(data.LocationAbbr == state) & (data.Question == "Amount of alcohol excise tax by beverage type (wine)")].tolist()))

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [212]:
import re

def generate_ngrams(s, n):
    s = s.lower()
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)S
    tokens = [token for token in s.split(" ") if token != ""]
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [283]:
string = "Data hackathons are super cool!"
generate_ngrams(string, 1)

['data', 'hackathons', 'are', 'super', 'cool']

In [293]:
grams_dict = {}
questions = data.Question.unique().tolist()

for question in questions:
    grams = generate_ngrams(question, 7)
    topic = data.loc[data.Question == question].Topic.tolist()[0] 
    for gram in grams:
        if gram in grams_dict and topic not in grams_dict[gram]:
            grams_dict[gram].append(topic)
        else:
            grams_dict[gram] = [topic]
            
grams_dict

{'amount of alcohol excise tax by beverage': ['Alcohol'],
 'of alcohol excise tax by beverage type': ['Alcohol'],
 'alcohol excise tax by beverage type wine': ['Alcohol'],
 'alcohol excise tax by beverage type beer': ['Alcohol'],
 'alcohol excise tax by beverage type distilled': ['Alcohol'],
 'excise tax by beverage type distilled spirits': ['Alcohol'],
 'hospitalization for heart failure among medicare eligible': ['Cardiovascular Disease'],
 'for heart failure among medicare eligible persons': ['Cardiovascular Disease'],
 'heart failure among medicare eligible persons aged': ['Cardiovascular Disease'],
 'failure among medicare eligible persons aged 65': ['Cardiovascular Disease'],
 'among medicare eligible persons aged 65 years': ['Chronic Obstructive Pulmonary Disease',
  'Older Adults'],
 'binge drinking frequency among adults aged 18': ['Alcohol'],
 'drinking frequency among adults aged 18 years': ['Alcohol'],
 'frequency among adults aged 18 years who': ['Alcohol'],
 'among adults

In [294]:
res = {}

for key in grams_dict:
    res[key] = len(grams_dict[key])
    
{k: v for k, v in sorted(res.items(), key=lambda item: item[1], reverse=True)}

#grams_dict['pregnancy']
    
# topics = grams_dict['influenza vaccination among']

# ans = []
# for topic in topx`ics:
#     questions = data.loc[data.Topic == topic].Question.unique().tolist()
#     for q in questions:
#         if 'influenza'.lower() in q.lower():
#             print(q)
    




{'influenza vaccination among noninstitutionalized adults aged 18': 4,
 'pneumococcal vaccination among noninstitutionalized adults aged 18': 4,
 'pneumococcal vaccination among noninstitutionalized adults aged 65': 4,
 'prevalence among women aged 18 44 years': 3,
 'vaccination among noninstitutionalized adults aged 65 years': 3,
 'among noninstitutionalized adults aged 65 years with': 3,
 'influenza vaccination among noninstitutionalized adults aged 65': 3,
 'among medicare eligible persons aged 65 years': 2,
 'visits to dentist or dental clinic among': 2,
 'to dentist or dental clinic among adults': 2,
 'dentist or dental clinic among adults aged': 2,
 'or dental clinic among adults aged 18': 2,
 'dental clinic among adults aged 18 years': 2,
 'vaccination among noninstitutionalized adults aged 18 64': 2,
 'among noninstitutionalized adults aged 18 64 years': 2,
 'noninstitutionalized adults aged 18 64 years with': 2,
 'unhealthy days among adults aged 18 years': 2,
 'of high blood 

In [3]:
def get_distribution(li):
    dict_ = {}
    for i in li:
        if not i in dict_:
            dict_[i] = 1
        else:
            dict_[i] += 1
    return dict_

stop_words = set(stopwords.words('english')) 
all_questions = ""
for question in data.Question.unique():
    all_questions += question
    all_questions += " "
word_tokens = word_tokenize(all_questions) 
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
res = get_distribution(filtered_sentence)
x = {k: v for k, v in sorted(res.items(), key=lambda item: item[1])}

words_to_cloud = ""
for word in filtered_sentence:
    words_to_cloud += word + " "
    
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = [], 
                min_font_size = 10).generate(words_to_cloud) 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.savefig("cloud.pdf")
plt.close()